In [ ]:
import os
import json 
img_list = os.listdir('path_to_images')
import torch
torch.save(img_list, 'path_to_save_image_list')

In [ ]:
with open('path_to_image_captions_in_test_set_jsonl', 'r') as json_file:
    json_list = list(json_file)

for json_str in json_list:
    result = json.loads(json_str)
    dev.append(result)
    
    

In [ ]:
train = []
with open('path_to_image_captions_in_train_set_jsonl', 'r') as json_file:
    json_list = list(json_file)

for json_str in json_list:
    result = json.loads(json_str)
    train.append(result)
    

##Training Without Fine-Tuning

In [ ]:
import pandas as pd
train_df = pd.DataFrame(train)
train_df.reset_index(drop=True, inplace=False)
train_df = train_df.sort_values(by=['id'])
train_df = train_df.reset_index(drop=True)

In [ ]:
import torch
import os
import numpy as np


def oscar_outputs_to_xgbdims(oscar_output_path):
  oscar_output = torch.load(oscar_output_path)
  vec1_list = []
  vec2_list = []
  label_list = []
  for entry_ids in list(oscar_output.keys()):
    oscar_entry = oscar_output[entry_ids]
    vec1 = oscar_entry['vector1']
    vec2 = oscar_entry['vector2'].squeeze(0)
    label = torch.tensor(oscar_entry['hateful']).unsqueeze(0)
    vec1_list.append(vec1)
    vec2_list.append(vec2)
    label_list.append(label)

  label_array = torch.cat(label_list)
  data_array1 = torch.stack(vec1_list)
  data_array2 = torch.stack(vec2_list)

  return label_array, data_array1, data_array2

In [ ]:
import torch
import numpy as np
import os
import xgboost as xgb
from sklearn.model_selection import train_test_split

label = []
data1 = []
data2 = []

greater_dir_path = 'path_to_train_oscar_tensors'

for output_name in os.listdir(greater_dir_path):
  output_path = os.path.join(greater_dir_path, output_name)
  l, d1, d2 = oscar_outputs_to_xgbdims(output_path)
  label.append(l)
  data1.append(d1)
  data2.append(d2)

label = torch.cat(label)
data1 = torch.vstack(data1)
data2 = torch.vstack(data2)

label = label.detach().numpy()
data1 = data1.detach().numpy()
data2 = data2.detach().numpy()

# X_train, X_test, y_train, y_test = train_test_split(
# data1, label, test_size=0.2, random_state=42)
# dtrain1 = xgb.DMatrix(X_train, label=y_train)

# params = {
#   'colsample_bynode': 0.8,
#   'learning_rate': 1,
#   'max_depth': 5,
#   'num_parallel_tree': 50,
#   'objective': 'binary:logistic',
#   'subsample': 0.8,
# }

# params['gpu_id'] = 0
# params['tree_method'] = 'gpu_hist'

# bst = xgb.train(params, dtrain1, num_boost_round=10)

# y_pred = bst.predict(xgb.DMatrix(X_test))


In [ ]:
from sklearn.metrics import f1_score

X_train, X_test, y_train, y_test = train_test_split(
data2, label, test_size=0.3, random_state=42)
dtrain1 = xgb.DMatrix(X_train, label=y_train)

params = {
  'colsample_bynode': 0.8,
  'learning_rate': 1,
  'max_depth': 10,
  'num_parallel_tree': 10,
  'objective': 'binary:logistic',
  'subsample': 0.8,
}

params['gpu_id'] = 0
params['tree_method'] = 'gpu_hist'

bst = xgb.train(params, dtrain1, num_boost_round=10)

y_pred = bst.predict(xgb.DMatrix(X_test))

y_pred = np.array([1 if y_pred[i] >= 0.5 else 0 for i in range(0, y_pred.shape[0])])
print('acc: '+str(accuracy_score(y_test, y_pred)))

print(f1_score(y_test, y_pred, average='micro'))

print(f1_score(y_test, y_pred, average='macro'))

In [ ]:
print(f1_score(y_test, y_pred, average='micro'))

print(f1_score(y_test, y_pred, average='macro'))

In [ ]:
import xgboost as xgb


In [ ]:
bst.predict(xgb.DMatrix(data1[3][np.newaxis, :])).item()

In [ ]:
from sklearn.metrics import accuracy_score


greater_dir_path = 'path_to_test_oscar_tensors'

dev_label = []
dev_data1 = []
dev_data2 = []

for output_name in os.listdir(greater_dir_path):
  output_path = os.path.join(greater_dir_path, output_name)
  l, d1, d2 = oscar_outputs_to_xgbdims(output_path)
  dev_label.append(l)
  dev_data1.append(d1)
  dev_data2.append(d2)

dev_label = torch.cat(dev_label)
dev_data1 = torch.vstack(dev_data1)
dev_data2 = torch.vstack(dev_data2)

dev_label = dev_label.detach().numpy()
dev_data1 = dev_data1.detach().numpy()
dev_data2 = dev_data2.detach().numpy()

dev_dtrain1 = xgb.DMatrix(dev_data1)

dev_pred = bst.predict(dev_dtrain1)
dev_pred = np.array([1 if dev_pred[i] >= 0.7 else 0 for i in range(0, dev_pred.shape[0])])

accuracy_score(dev_pred, dev_label)